In [1]:
import sys
import subprocess
import re
import os
import time
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cmath
import operator
import random
from operator import itemgetter, attrgetter
import matplotlib.cm as cm
import vtk

## import MI measures
from mi_functions import *


In [8]:
# isabel
file1 = '../Data/Isabel_vti/isabel_p_25.vti'
file2 = '../Data/Isabel_vti/isabel_qva_25.vti'
arrayName1 = 'Pressure'
arrayName2 = 'QVapor'
data_set = 'isabel'
samp_type = 'regular'


# # ## Asteroid
# file1 = '../Data/Asteroid/tev.vti'
# file2 = '../Data/Asteroid/v02.vti'
# arrayName1 = 'tev'
# arrayName2 = 'v02'
# data_set = 'asteroid'
# samp_type = 'random' # random, pmi


# ## Nyx: high res
# file1 = '/Users/sdutta/Data/Nyx/Density_small.vti'
# file2 = '/Users/sdutta/Data/Nyx/Temperature_small.vti'
# arrayName1 = 'Density'
# arrayName2 = 'Temperature'
# outfile = '../output/joint_pmi_sampled_nyx.vtp'
# pmivol_file = '../output/pmi_vol_dens_temp_nyx.vti'

# ## Nyx
# file1 = '../Data/Nyx/density_1.vti'
# file2 = '../Data/Nyx/temperature_1.vti'
# arrayName1 = 'logField'
# arrayName2 = 'logField'
# outfile = '../output/joint_pmi_sampled_nyx.vtp'
# pmivol_file = '../output/pmi_vol_dens_temp_nyx.vti'

# ## Combustion
# file1 = '../Data/Combustion/combustion_mixfrac.vti'
# file2 = '../Data/Combustion/combustion_chi.vti'
# arrayName1 = 'mixfrac'
# arrayName2 = 'chi'
# outfile = '../output/joint_pmi_sampled_combustion.vtp'
# pmivol_file = '../output/pmi_vol_mixfrac_chi.vti'

# ## Mfix
# file1 = '../Data/Mfix/original_density_out.vti'
# file2 = '../Data/Mfix/gradient_out.vti'
# arrayName1 = 'ImageScalars'
# arrayName2 = 'ImageScalars'
# outfile = '../output/joint_pmi_sampled_mfix.vtp'
# pmivol_file = '../output/pmi_vol_gradient_density.vti'

In [9]:
# ## load vti data
R1 = vtk.vtkXMLImageDataReader()
R1.SetFileName(file1)
R1.Update()
dataArray1 = R1.GetOutput().GetPointData().GetArray(arrayName1)

R2 = vtk.vtkXMLImageDataReader()
R2.SetFileName(file2)
R2.Update()
dataArray2 = R2.GetOutput().GetPointData().GetArray(arrayName2)

var1=np.zeros(dataArray1.GetNumberOfTuples()) 
var2=np.zeros(dataArray2.GetNumberOfTuples()) 

for i in range(dataArray1.GetNumberOfTuples()):
    var1[i] = dataArray1.GetTuple1(i)
    var2[i] = dataArray2.GetTuple1(i)

dims = R1.GetOutput().GetDimensions()

min_var1 = np.min(var1)
max_var1 = np.max(var1)

min_var2 = np.min(var2)
max_var2 = np.max(var2)

print R1.GetOutput().GetSpacing()
print R1.GetOutput().GetOrigin()

(1.0, 1.0, 1.0)
(0.0, 0.0, 0.0)


In [10]:
totPercentage = 1

In [11]:
def compute_3d_to_1d_map (pts, dims):
    return pts[0] + dims[0]*(pts[1]+dims[1]*pts[2])

####################################################################
def insert_corner_pts(data,arrayName1,data1,arrayName2,sampled_data):
    
    dims = data.GetDimensions()
    varArr = data.GetPointData().GetArray(arrayName1)
    varArr1 = data1.GetPointData().GetArray(arrayName2)
    
    pts = [0,0,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [0,dims[1]-1,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [0,0,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [0,dims[1]-1,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [dims[0]-1,0,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [dims[0]-1,dims[1]-1,0]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [dims[0]-1,0,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    pts = [dims[0]-1,dims[1]-1,dims[2]-1]
    idx = compute_3d_to_1d_map(pts,dims)
    pts = data.GetPoint(idx)
    var_val = varArr.GetTuple1(idx)
    var_val1 = varArr1.GetTuple1(idx)
    sampled_data.GetPoints().InsertNextPoint(pts)
    sampled_data.GetPointData().GetArray(arrayName1).InsertNextTuple1(var_val)
    sampled_data.GetPointData().GetArray(arrayName2).InsertNextTuple1(var_val1)
    
    return sampled_data

######################################################################
def sample_regular(data,arrayName1,data1,arrayName2,percentage):
    
    pdata = vtk.vtkPolyData()
    pts_arr = vtk.vtkPoints()
    sampled_arr = vtk.vtkDoubleArray()
    sampled_arr.SetName(arrayName1)
    sampled_arr1 = vtk.vtkDoubleArray()
    sampled_arr1.SetName(arrayName2)
    
    totPts = data.GetNumberOfPoints()
    numOfPtsNeeded = int((totPts*percentage)/100.0)
    print 'samples needed: ' + str(numOfPtsNeeded)

        
    pdata.SetPoints(pts_arr)
    pdata.GetPointData().AddArray(sampled_arr)
    pdata.GetPointData().AddArray(sampled_arr1)
    
    return pdata

######################################################################
def write_vtp(filename,pdata):
    pdatawriter = vtk.vtkXMLPolyDataWriter()
    pdatawriter.SetFileName(filename)
    pdatawriter.SetInputData(pdata)
    pdatawriter.Write()

In [12]:
sampled_data = sample_regular(R1.GetOutput(),arrayName1,R2.GetOutput(),arrayName2,totPercentage)

samples needed: 31250


In [12]:
from scipy.spatial import Delaunay
points = np.array([[0, 0], [0, 1.1], [1, 0], [1, 1]])
tri = Delaunay(points)

print tri.convex_hull


[[2 0]
 [2 3]
 [1 0]
 [3 1]]
